## Create a simple Define-XML v2.1 file
First make sure you've installed odmlib using pip:
```
pip install odmlib
```
Next import the Define-XML v2.1 model. We'll also use the datetime module in this example as well.

In [17]:
import odmlib.define_2_1.model as DEFINE
import datetime

Next, we'll create the ODM object. There's no need to include namespaces in the ODM object since these are
managed by odmlib.

In [18]:
current_datetime = datetime.datetime.utcnow().replace(tzinfo=datetime.timezone.utc).isoformat()
odm = DEFINE.ODM(FileOID="DEF.COSA.DEMO",
              AsOfDateTime=current_datetime,
              CreationDateTime=current_datetime,
              ODMVersion="1.3.2",
              FileType="Snapshot",
              Originator="Sam Hume",
              SourceSystem="odmlib",
              SourceSystemVersion="0.1.4",
              Context="Other")

Now we've created the ODM object. Next we'll instantiate a Study object and assign it to odm.Study. In the ODM model Study is a list, but in Define-XML it is a singular object.

Then we'll add the global variables to Study. Since there's only one GlobalVariables object we just use an assignment
rather than append. We'll also add the individual global variable objects. In the ODM specification, the global
variables, like StudyName, have values in the body of the element. In odmlib, you assign values to the body of the
element using _content as shown below.

In [19]:
study = DEFINE.Study(OID="ST.DEFINE.COSA.001")
study.GlobalVariables.StudyName = DEFINE.StudyName(_content="TEST Define-XML ItemGroupDef")
study.GlobalVariables.StudyDescription = DEFINE.StudyDescription(_content="ItemGroupDef 001")
study.GlobalVariables.ProtocolName = DEFINE.ProtocolName(_content="Define-XML ItemGroupDef")
odm.Study = study

We created the Study object and assigned it to ODM Study.

In [20]:
mdv = DEFINE.MetaDataVersion(OID="MDV.COSA.IGD.001", Name="ItemGroupDefDemo001",
                                     Description="ItemGroupDef COSA Demo", DefineVersion="2.1.0")

Next creating Standards

In [21]:
mdv.Standards.Standard.append(DEFINE.Standard(OID="STD.1", Name="SDTMIG", Type="IG", Version="3.2", Status="Final"))
mdv.Standards.Standard.append(DEFINE.Standard(OID="STD.2", Name="CDISC/NCI", Type="CT", PublishingSet="SDTM",
                                              Version="2021-12-17", Status="Final"))

Now create an ItemGroupDef object and populate all the attributes.

In [22]:
igd = DEFINE.ItemGroupDef(OID="IG.VS",
                          Name="VS",
                          Repeating="Yes",
                          Domain="VS",
                          SASDatasetName="VS",
                          IsReferenceData="No",
                          Purpose="Tabulation",
                          ArchiveLocationID="LF.VS",
                          Structure="One record per vital sign measurement per visit per subject",
                          StandardOID="STD.1",
                          IsNonStandard="Yes",
                          HasNoData="Yes")

Using the igd ItemGroupDef object we can create and add the necessary child elements.

In [23]:
igd.Description.TranslatedText.append(DEFINE.TranslatedText(_content="Vital Signs", lang="en"))
igd.ItemRef.append(DEFINE.ItemRef(ItemOID="IT.STUDYID", Mandatory="Yes", OrderNumber=1, KeySequence=1))
igd.ItemRef.append(DEFINE.ItemRef(ItemOID="IT.VS.DOMIAN", Mandatory="Yes", OrderNumber=2))
igd.ItemRef.append(DEFINE.ItemRef(ItemOID="IT.USUBJID", Mandatory="Yes", OrderNumber=3, KeySequence=2))
igd.ItemRef.append(DEFINE.ItemRef(ItemOID="IT.VS.VSSEQ", Mandatory="Yes", OrderNumber=4))
igd.ItemRef.append(DEFINE.ItemRef(ItemOID="IT.VS.VSTESTCD", Mandatory="Yes", OrderNumber=5, KeySequence=3))
igd.ItemRef.append(DEFINE.ItemRef(ItemOID="IT.VS.VSTEST", Mandatory="Yes", OrderNumber=6))

Before we add the remaining objects, let's show what happens when we try to create an object using invalid inputs.
If you attempt to create an invalid ItemRef odmlib will return an error. There are a number of ways that odmlib ensures
quality or checks for quality, but most of them are beyond what we'll cover in this demo.

In [24]:
try:
    ir = DEFINE.ItemRef(Mandatory="Yes", OrderNumber=1)
except ValueError as ve:
    print(f"Error creating ItemRef: {ve}")


Error creating ItemRef: Missing required keyword argument ItemOID in ItemRef


That's enough ItemRefs for now. Next let's add the new Class element.

In [25]:
igd.Class = DEFINE.Class(Name="FINDINGS")

Now that the ItemGroupDef has been defined, we need to add it to the MetaDataVersion object so that it will be included
in the Define-XML document. Note that because ItemGroupDef is a list we append instantiated item groups.

In [26]:
odm.Study.MetaDataVersion = mdv
odm.Study.MetaDataVersion.ItemGroupDef.append(igd)

In [27]:
odm.write_xml(odm_file="./data/cosa_define_demo.xml")

Next, we'll take a look at the ODM text.

In [28]:
with open("./data/cosa_define_demo.xml", 'r') as file:
    cosa_odm = file.read()
print(cosa_odm)

<?xml version='1.0' encoding='utf-8'?>
<ODM FileOID="DEF.COSA.DEMO" AsOfDateTime="2022-03-14T13:23:47.644445+00:00" CreationDateTime="2022-03-14T13:23:47.644445+00:00" ODMVersion="1.3.2" FileType="Snapshot" Originator="Sam Hume" SourceSystem="odmlib" SourceSystemVersion="0.1.4" def:Context="Other" xmlns="http://www.cdisc.org/ns/odm/v1.3" xmlns:xs="http://www.w3.org/2001/XMLSchema-instance" xmlns:xml="http://www.w3.org/XML/1998/namespace" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:def="http://www.cdisc.org/ns/def/v2.1"><Study OID="ST.DEFINE.COSA.001"><GlobalVariables><StudyName>TEST Define-XML ItemGroupDef</StudyName><StudyDescription>ItemGroupDef 001</StudyDescription><ProtocolName>Define-XML ItemGroupDef</ProtocolName></GlobalVariables><MetaDataVersion OID="MDV.COSA.IGD.001" Name="ItemGroupDefDemo001" Description="ItemGroupDef COSA Demo" def:DefineVersion="2.1.0"><def:Standards><def:Standard OID="STD.1" Name="SDTMIG" Type="IG" Version="3.2" Status="Final" /><def:Standard OID="ST

Now try to validate the Define-XML file using the Define-XML v2.1 XML schemas.

In [29]:
from odmlib import odm_parser as P
# schema_file = "C:\\Users\\SamHume\\schema\\cdisc-define-2.1\\define2-1-0.xsd"
schema_file = "/home/sam/standards/DefineV211/schema/cdisc-define-2.1/define2-1-0.xsd"

validator = P.ODMSchemaValidator(schema_file)
try:
    validator.validate_file("./data/cosa_define_demo.xml")
    print("define-XML schema validation completed successfully...")
except P.OdmlibSchemaValidationError as ve:
    print(f"schema validation errors: {ve}")

define-XML schema validation completed successfully...


Now that we've created a simple Define-XML file, we'll look at how to load the file into odmlib. First, we need to
import the Define-XML loader. Then we'll load the Define-XML file we just created.

In [14]:
from odmlib import define_loader as DL, loader as LD
loader = LD.ODMLoader(DL.XMLDefineLoader(model_package="define_2_1", ns_uri="http://www.cdisc.org/ns/def/v2.1"))
loader.open_odm_document("./data/cosa_define_demo.xml")

<Element '{http://www.cdisc.org/ns/odm/v1.3}ODM' at 0x7fccc0132590>

Now that the Define-XML file is loaded into odmlib we can print out some of the content.

In [15]:
odm = loader.load_odm()
print(f"Study OID is {odm.Study.OID}")
print(f"Study Name is {odm.Study.GlobalVariables.StudyName}")
print(f"Study Description is {odm.Study.GlobalVariables.StudyDescription}")
print(f"Protocol Name is {odm.Study.GlobalVariables.ProtocolName}")

Study OID is ST.DEFINE.COSA.001
Study Name is TEST Define-XML ItemGroupDef
Study Description is ItemGroupDef 001
Protocol Name is Define-XML ItemGroupDef


You could also generate a Python Dictionary or JSON instead of creating an XML ODM document.

In [16]:
cosa_dict = odm.to_dict()
print(cosa_dict)

{'FileOID': 'DEF.COSA.DEMO', 'AsOfDateTime': '2022-03-14T13:23:37.912770+00:00', 'CreationDateTime': '2022-03-14T13:23:37.912770+00:00', 'ODMVersion': '1.3.2', 'FileType': 'Snapshot', 'Originator': 'Sam Hume', 'SourceSystem': 'odmlib', 'SourceSystemVersion': '0.1.4', 'Context': 'Other', 'Study': {'OID': 'ST.DEFINE.COSA.001', 'GlobalVariables': {'StudyName': {'_content': 'TEST Define-XML ItemGroupDef'}, 'StudyDescription': {'_content': 'ItemGroupDef 001'}, 'ProtocolName': {'_content': 'Define-XML ItemGroupDef'}}, 'MetaDataVersion': {'OID': 'MDV.COSA.IGD.001', 'Name': 'ItemGroupDefDemo001', 'Description': 'ItemGroupDef COSA Demo', 'DefineVersion': '2.1.0', 'Standards': {'Standard': [{'OID': 'STD.1', 'Name': 'SDTMIG', 'Type': 'IG', 'Version': '3.2', 'Status': 'Final'}, {'OID': 'STD.2', 'Name': 'CDISC/NCI', 'Type': 'CT', 'PublishingSet': 'SDTM', 'Version': '2021-12-17', 'Status': 'Final'}]}, 'ItemGroupDef': [{'OID': 'IG.VS', 'Name': 'VS', 'Repeating': 'Yes', 'Domain': 'VS', 'SASDatasetName

Although it's an incomplete Define-XML, there's enough content to validate successfully. That concludes this simple
Define-XML v2.1 odmlib demo.
